In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import scipy.stats

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Final_data/test.csv')
samplesubmission = pd.read_csv('/content/drive/MyDrive/Final_data/sample_submission.csv')

In [ ]:
test_df.shape

(20775, 25)

In [ ]:
test_df.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044


In [ ]:
from sklearn.preprocessing import LabelEncoder
import warnings
from pandas.core.common import SettingWithCopyWarning
from joblib import load
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)



# Label encode the categorical features
le = LabelEncoder()
for col in ['attribute_0', 'attribute_1']:
    le.fit(test_df[col])
    test_df[col] = le.transform(test_df[col])



test_df['m3_flag'] = test_df.measurement_3.isna()
test_df['m5_flag'] = test_df.measurement_5.isna()


# Fill in missing values
features = []
for feature in test_df.columns:
    if feature == 'loading' or feature.startswith('measurement'):
        features.append(feature)
imputer = KNNImputer(n_neighbors=15)
imputer.fit(test_df[features])
test_df[features] = imputer.transform(test_df[features])



# Clip values of measurement_2
test_df['measurement_2'] = test_df['measurement_2'].clip(11, None)

features2 = ['loading', 'attribute_3', 'measurement_2', 'measurement_4', 'measurement_17', 'm3_flag', 'm5_flag']#, 'm_3_missing', 'm_5_missing'


#model = LogisticRegression()
#model = make_pipeline(StandardScaler(), LogisticRegression())
model = make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', C=0.01,solver='liblinear', random_state=1,class_weight='balanced'))
model = load('/content/drive/MyDrive/Final_data/model.joblib')



y_pred = model.predict_proba(test_df[features2])[:,1]





# saving result
samplesubmission['failure'] = y_pred
samplesubmission.to_csv('/content/drive/MyDrive/Final_data/submission.csv', index=False)
